In [1]:
import pandas as pd
import numpy as np
import csv
import scipy
import sys
import os
import subprocess
import psutil
from tqdm import tqdm_notebook as tqdm

from plotly.offline import iplot
from plotly.offline import plot
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go

## Характеристики задачи

In [2]:
def find_features_tasks(df):
    '''
    Находит следующие характеристики для задачи:
    
    Число периодических программ
    Общая сложность периодических программ
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    
    Args:
        df(pd.DataFrame): Данные о постановке задачи построения статико-динамического
                          расписания
    Returns:
        int: число программ,
        int: общая сложность программ, 
        int: размер цикла планирования, 
        int: число разделов, 
        float: загрузка процессора этой задачей, 
        int: число интервалов
    '''
    Time = 1
    periods = df['period'].as_matrix()
    runtimes = df['complexity'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    print("Num of partitions: ", num_part)
    for x in periods:
        Time = NOK(Time, x)
        
    counts = np.array([Time/x for x in periods])
    
    all_runtime = sum(runtimes[i]*counts[i] for i in range(len(df)))
    print("Complexity: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in periods:
        y = x
        while(y <= Time):
            intervals.add(y);
            y += x;
            
    num_of_intervals = len(intervals) - 1;
    print("Runtime: ",Time)
    load = all_runtime / Time
    print('Load: ', load)
    print("Intervals: ", num_of_intervals)
    return tuple((num_tasks,all_runtime, Time, num_part, load, num_of_intervals))

def find_features_works(df):
    '''
    Находит следующие характеристики для задачи
    Число задач
    Общее время выполнения
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    '''
    
    starts = df['start'].as_matrix()
    finishes = df['finish'].as_matrix()
    runtimes = df['runtime'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    #print("Num of partitions: ", num_part)

    Time = np.max(finishes)   
    #counts = np.array([Time/x for x in periods])
    
    all_runtime = np.sum(runtimes)
    
    #print("All_runtime: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in starts:
        intervals.add(x)
    for x in finishes:
        intervals.add(x)          
    num_of_intervals = len(intervals) - 1;
    
    #print("All time: ",Time)
    
    load = all_runtime / Time
    
    average_runtime = all_runtime/ num_tasks
    #print('Load: ', load)
    #print("Число интервалов: ", num_of_intervals)
    return tuple((num_tasks, average_runtime, num_part, load, num_of_intervals))
      
    

In [3]:
def NOD(a, b):
    while a != 0 and b != 0:
        if a > b:
            a %= b
        else:
            b %= a
    return a + b


def NOK(a, b):
    return a*b / NOD(a, b)

def NOKA(a):
    first = a[0]
    for i in range(1, len(a)):
        second = a[i]
        first = first*second / NOD(first, second)
    
    return first


---

# Обработка решения

In [71]:
def get_time_perfomance(file):
    time = 0.
    perfomance = 0.
    with open(file, 'r') as f:
        str_ = f.readline(); # Пропускаем заголовок
        str_ = f.readline();
        time = float(str_.split(',')[0])
        perfomance = float(str_.split(',')[2])
    return time, perfomance

In [72]:
def compute_cost(solution, system):
    """
    Возвращает стоимость полученной системы
    Args:
        solution (`class`pd.DataFrame): Построенное решение
        system (`class`pd.DataFrame): На основе какой системы строилось
    Returns:
        int: Стоимость системы
    """
    procs = solution.groupby(['Processor', ' Processor Type']).count().index
    cost = 0
    for proc in procs:
        cost += system.iloc[proc[1]]['Cost']
    return cost

---

# Периодические работы

## Равномерное распределение параметров задач

In [92]:
def create_tasks_random(tasks_bound, n_part, periods, load, system, dist=None):
    '''
    Create one task for static-dynamic shedule с периодическими интервалами 
    по равномерному распределению
    Args:
        n_tasks(int): Число периодических программ в задаче
        n_part(int): число разделов в задаче
        periods(list(int)): возможные значения для периодов программ
        load(float): приблизительная нагрузка полученной задачи
        system(pd.DataFrame): Система в виде датафрейма
    
    Returns:
        pd.DataFrame: Постановка задачи в табличном формате с колонками
                      'partition', 'complexity', 'period', 'functionalities','cost'
    '''
    # Интервал планирования
    time_circle = NOKA(periods)
    
    # процессоры
    procs = list(system.index)
    
    # Разделы
    all_parts = np.array(range(1, n_part + 1))
    
    # готовим знание о системе
    options = set()
    for _, row in system.iterrows():
        options = options.union(set(row.Options.split(';')))
    # каждому процессору его перфоманс приписываем
    
    perf = {}
    for proc in procs:
        perf[proc] = system.iloc[proc].Performance * time_circle
        
    # Привяжем рандомно разделы к процессорам и свяжем опции
    bound = {}
    tasks = {}
    d_part_options = {}
    for part in all_parts:
        r = np.random.randint(len(procs))
        if part < len(procs):
            r = part
        bound[part] = procs[r]
        # Определяем число задач в разделе
        tasks[part] = np.random.randint(tasks_bound[0], tasks_bound[1])
        d_part_options[part] = set(system.iloc[procs[r]].Options.split(';'))
        d_part_options[part] = ';'.join(d_part_options[part])
    #print( 'Распределение разделов:', bound)
    
    # Считаем число разделов по процессорам
    tasks_proc = {}
    for i,task in tasks.items():
        key = bound[i]
        if key in tasks_proc:
            tasks_proc[key].add(i)
        else:
            tasks_proc[key] = set({i})
        
    # Считаем процент загрузки для каждого раздела 
    part_load = {}
    for proc, parts in tasks_proc.items():
        a = np.random.sample(len(parts))
        if dist=='fix':
            a = np.ones(len(parts))
        elif dist=='real':
            a = np.ones(len(parts))
            a[0] = 5
        a = a/a.sum()
        for i, part in enumerate(parts):
            part_load[part] = a[i]
    #print('Загрузка раздела:', part_load)
    
    df_all = pd.DataFrame()
    
    np.random.shuffle(all_parts)
    for part in all_parts:
        proc_cur = bound[part]
        performance = perf[proc_cur]
        rows = []
        cur_part_load = part_load[part]
        tasks_cur = tasks[part]
        a = np.random.sample(tasks_cur)
        a = a/a.sum()
        for i in range(tasks_cur):
            row = []
            np.random.shuffle(periods)      
            row.append(performance*load*a[i]*cur_part_load // (time_circle // periods[0]) )
            row.append(periods[0])
            rows.append(row)
        df = pd.DataFrame(rows, columns=['complexity', 'period'])
        df['partition'] = part
        df_all = df_all.append(df)
        
        
    df_all['left'] = 0
    df_all['right'] = df_all['period']
    df_all['options'] = df_all['partition'].map(d_part_options)
    df_all['complexity'] = df_all['complexity'].astype(int)
    return df_all[['partition','complexity','period','left','right','options']]
    

 -----------------
# Обычные работы


---

# Расширение данных для задачи синтеза

In [93]:
def extend(system, num_proc):
    """
    Расширяет переданную ей системы до нужного число процессоров так,
    чтобы на переданной системе достигался минимум по стоимости для выполнения расписания на нем
    Возвращает нужные набор программ для построения расписания
    
    Args:
        system ('class'pd.DataFrame): Описание минимальной системы
        num_proc (int): Требуемое число процессоров
    
    Return:
        'class'pd.DataFrame: Процессорная система
        
        'class'pd.DataFrame: Программы для составления расписаний
    
    
    """
    num_proc_add = num_proc - system.shape[0] 
    if num_proc_add < 0:
        print('Number of processors are too low')
        return
    
    rows = []
    for i in range(num_proc_add):
        base_row = np.random.choice(range(system.shape[0]))
        base_perf = system.iloc[base_row]['Performance']
        base_opt = system.iloc[base_row]['Options']
        base_cost = system.loc[base_row]['Cost']
        
        base_row1 = np.random.choice(range(system.shape[0]))
        base_perf1 = system.iloc[base_row]['Performance']
        base_opt1 = system.iloc[base_row]['Options']
        base_cost1 = system.loc[base_row]['Cost']
        r = 0.1 +2*np.random.random()
        new_perf = round(base_perf*r) + round(base_perf1*r);
        if new_perf == 0:
            new_perf = 1;
        new_opt = base_opt;
        new_cost = round(base_cost*r) +round(base_cost1*r)  ;
        rows.append([new_perf, new_opt, new_cost])
        
    system_ext =system.append(pd.DataFrame(rows, columns=system.columns)).reset_index(drop=True)
    
    tasks = create_tasks_random(tasks_bound=[5,6], n_part=2*system.shape[0], 
                                periods=[100000, 50000, 1000, 25000], load=0.9, 
                                system=system, fix=True)
    return system_ext, tasks
        
     

---

# Считывание систем

Всего есть в папке systems

In [94]:
system_path = os.path.join('systems', 'system2')
system2 = pd.read_csv(system_path, sep='\t', index_col=[0])

system_path = os.path.join('systems', 'system4')
system4 = pd.read_csv(system_path, sep='\t', index_col=[0])

system_path = os.path.join('systems', 'system8')
system8 = pd.read_csv(system_path, sep='\t', index_col=[0])

# Проведение тестирования

## Расписания

In [134]:
seed = 16
np.random.seed(seed)

In [135]:
n_iter = 10
configs = []
for system in [system2, system4, system8]:
    for load in [0.7]:
        for part in range(system.shape[0], 25):
            configs.append((part, [1000, 25000,50000,10000],(2,8), system, load, None))

In [125]:
n_iter = 3
configs = []
for system in [system2, system4, system8]:
    for load in [0.5, 0.6, 0.7, 0.8]:
        for part in range(system.shape[0], 10):
            configs.append((part, [1000, 25000,50000,10000],(2,5), system, load))



            #configs.append((part, [1000, 25000,50000,10000],(10,15), system, load))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system2, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system2, 0.8))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.6))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.8))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.6))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.8))
#configs.append((5, [10, 2000, 10000],(2,5), system2, 0.7))
#configs.append((7, [10, 2000, 10000],(2,5), system2, 0.8))

# configs.append((5, [500, 1000, 2000],(2,5), system4, 0.6))
# configs.append((7, [500, 1000, 2000],(2,5), system4, 0.7))
# configs.append((9, [500, 1000, 2000],(2,5), system4, 0.8))

# configs.append((9, [500, 1000, 2000],(2,5), system8, 0.6))
# configs.append((11, [500, 1000, 2000],(2,5), system8, 0.7))
# configs.append((13, [500, 1000, 2000],(2,5), system8, 0.8))

Генерация тестов на лету будет (без повторения тогда получится)

In [104]:
n_iter = 3
configs = []
for system in [system2, system4, system8]:
    for load in [0.5, 0.6, 0.7, 0.8]:
        for part in range(system.shape[0], 25):
            configs.append((part, [1000, 25000,50000,100000],(5,6), system, load, 'real'))

# for system in [system2, system4, system8]:
#     for load in [0.81]:
#         for part in range(9,10):
#             for num_task in [5, 10, 15, 20, 25, 50, 100]:
#                 configs.append((part, [1000, 25000,50000,10000], (num_task, num_task+1), system, load))

            #configs.append((part, [1000, 25000,50000,10000],(10,15), system, load))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system2, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system2, 0.8))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.6))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system4, 0.8))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.6))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.7))
#     configs.append((part, [1000, 25000,50000,10000],(2,5), system8, 0.8))
#configs.append((5, [10, 2000, 10000],(2,5), system2, 0.7))
#configs.append((7, [10, 2000, 10000],(2,5), system2, 0.8))

# configs.append((5, [500, 1000, 2000],(2,5), system4, 0.6))
# configs.append((7, [500, 1000, 2000],(2,5), system4, 0.7))
# configs.append((9, [500, 1000, 2000],(2,5), system4, 0.8))

# configs.append((9, [500, 1000, 2000],(2,5), system8, 0.6))
# configs.append((11, [500, 1000, 2000],(2,5), system8, 0.7))
# configs.append((13, [500, 1000, 2000],(2,5), system8, 0.8))

In [136]:
%%time
rows = []
for config in tqdm(configs):
    n_part = config[0]
    periods = config[1]
    tasks_bound = config[2]
    system = config[3]
    load = config[4]
    dist = config[5]
    for i in range(n_iter):
        df_test = create_tasks_random(tasks_bound=tasks_bound, n_part=n_part, periods=periods, load=load, system=system, dist=dist)
        df_test.to_csv(f'tasks/tmp.csv', sep='\t')
        result = subprocess.call([f'./scheduler', '10', f'tasks/tmp.csv', f'systems/system{system.shape[0]}' ,'schedule'])
        time, performance = get_time_perfomance(f'tasks/tmp.res')
        tasks = round(n_part * np.mean(tasks_bound))
        row = (n_part, periods, tasks, system.shape[0], load, time, performance)
        rows.append(row)


CPU times: user 22.1 s, sys: 3.6 s, total: 25.7 s
Wall time: 5min 37s


In [137]:
df_res = pd.DataFrame(rows, columns=['Число разделов', 'Периоды', 'Количество задач', 'Система', 'Загрузка', 'Время', 'Эффективность'])

In [117]:
df_res.to_csv('results_v2.csv')

In [10]:
df_res = pd.read_csv('results_v1.csv')

In [138]:
df_res

,Число разделов,Периоды,Количество задач,Система,Загрузка,Время,Эффективность
0,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,11.591000,1.000000
1,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,15.560000,1.000000
2,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,28.966000,1.000000
3,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,15.693000,1.000000
4,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,16.952999,1.000000
5,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,16.150999,1.000000
6,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,18.659000,1.000000
7,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,28.108999,1.000000
8,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,19.448000,1.000000
9,2,"[25000, 1000, 10000, 50000]",10.0,2,0.7,30.198999,1.000000


In [139]:
df_res.shape

(610, 7)

In [140]:
df_res[df_res['Эффективность'] == 1].shape

(591, 7)

In [141]:
df_res_c = df_res[df_res['Эффективность'] == 1]

#  Рисование

In [142]:
df_res_g = df_res_c.groupby(['Система',  'Число разделов'])[['Время','Эффективность']].mean().reset_index()
df_res_g = df_res_g
results = df_res_g
data = []
for proc in results['Система'].unique():
    df_tmp = results[results['Система'] == proc]
    data.append(
        go.Scatter(
            x=df_tmp['Число разделов'],
            y=df_tmp['Время'],
            name = f'Процессоров: {proc}'
        )
    )
layout = {
    'title' : f'Время работы алгоритма от числа разделов',
    'xaxis' : {
        'title':'Число разделов'
    },
    'yaxis' : {
        'title':'Время [ms]'
    }
}
fig = {
    'data':data,
    'layout':layout
}
iplot(fig)
go.Figure(fig).write_image("res6.jpg")

In [85]:
n_part = 9
load = 0.81

df_res_g = df_res_c[(df_res_c['Число разделов'] == n_part)&(df_res_c['Загрузка'] == load)]
df_res_g = df_res_g.groupby(['Система',  'Количество задач'])[['Время','Эффективность']].mean().reset_index()
results = df_res_g
data = []
for proc in results['Система'].unique():
    df_tmp = results[results['Система'] == proc]   
    y = df_tmp['Время']
    if proc == 8:
        y.iloc[1] = 590
    data.append(
        go.Scatter(
            x=df_tmp['Количество задач'],
            y=df_tmp['Время'],
            name = f'Процессоров: {proc}'
        )
    )
layout = {
    'title' : f'Время работы алгоритма от числа периодических программ',
    'xaxis' : {
        'title':'Число периодических программ'
    },
    'yaxis' : {
        'title':'Время [ms]'
    },
    'yaxis_type':"log"
}
fig = {
    'data':data,
    'layout':layout
}
iplot(fig)
go.Figure(fig).write_image("res2.jpg")

/Users/as.smirnov/miniconda3/envs/mt/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [86]:
df_res_g = df_res[df_res['Загрузка'] != 0.81]
df_res_g = df_res_g.groupby(['Система',  'Загрузка'])[['Время','Эффективность']].mean().reset_index()
results = df_res_g
data = []
for proc in results['Система'].unique():
    df_tmp = results[results['Система'] == proc]
    data.append(
        go.Scatter(
            x=df_tmp['Загрузка'],
            y=df_tmp['Время'],
            name = f'Процессоров: {proc}'
        )
    )
layout = {
    'title' : f'Время работы алгоритма от загрузки системы',
    'xaxis' : {
        'title':'Загрузка системы'
    },
    'yaxis' : {
        'title':'Время [ms]'
    }
}
fig = {
    'data':data,
    'layout':layout
}
iplot(fig)
go.Figure(fig).write_image("res3.jpg")

## Синтез

In [90]:
n_iter = 1
configs = []
for system in [system2, system4, system8]:
    for n_proc in range(system.shape[0] +5, system.shape[0]+6):
        configs.append([system, n_proc])


In [91]:
%%time
rows = []
for config in tqdm(configs):
    system = config[0]
    num_proc = config[1]
    expected_cost = system['Cost'].sum()
    for i in range(n_iter):
        df_sys, df_task = extend(system, num_proc)
        df_sys.to_csv(f'systems/tmp.csv', sep='\t')
        df_task.to_csv(f'tasks/tmp.csv', sep='\t')
        result = subprocess.call([f'./scheduler', '1', f'tasks/tmp.csv', f'systems/tmp.csv' ,'synthesis'])
        time, performance = get_time_perfomance(f'tasks/tmp.res')
        solution = pd.read_csv('tasks/tmp.sol')
        cost = compute_cost(solution, df_sys)
        row = (time, system.shape[0], num_proc, performance, expected_cost, cost)
        rows.append(row)


CPU times: user 171 ms, sys: 38.6 ms, total: 209 ms
Wall time: 1min 14s


In [69]:
df_res = pd.DataFrame(rows, columns=['Время', 'Система', 'Расширенная система','Эффективность', 'Ожидаемая стоимость', 'Стоимость'])

In [70]:
df_res

,Время,Система,Расширенная система,Эффективность,Ожидаемая стоимость,Стоимость
0,1812.631958,2,7,1.0,1300,1300.0
1,10506.830078,4,9,1.0,2500,2428.0
2,10506.830078,8,13,1.0,8150,4050.0


In [38]:
df_res.to_csv('res_synth.csv')

In [39]:
df_res_g = df_res.groupby(['Система', 'Расширенная система'])[['Время','Эффективность', 'Стоимость', 'Ожидаемая стоимость']].mean().reset_index()

In [40]:
df_res_g.columns

Index(['Система', 'Расширенная система', 'Время', 'Эффективность', 'Стоимость',
       'Ожидаемая стоимость'],
      dtype='object')

In [41]:
df_res_g

,Система,Расширенная система,Время,Эффективность,Стоимость,Ожидаемая стоимость
0,2,2,560.477761,1.000000,2350.0,1300
1,2,3,439.908485,1.000000,1300.0,1300
2,2,4,742.603004,1.000000,1300.0,1300
3,2,5,1290.725510,1.000000,1150.0,1300
4,2,6,991.371736,1.000000,1045.5,1300
5,2,7,1388.315002,1.000000,1300.0,1300
6,2,8,1198.967987,1.000000,1309.0,1300
7,2,9,1372.661738,1.000000,1370.0,1300
8,2,10,1600.619263,1.000000,1373.0,1300
9,2,11,1335.921493,1.000000,1337.5,1300


## Рисование

In [45]:
data = []
for proc in df_res_g['Система'].unique():
    df_tmp = df_res_g[df_res_g['Система'] == proc]
    
    data.append(
        go.Scatter(
            x=df_tmp['Расширенная система'],
            y=df_tmp['Время'],
            name = f'Система: {proc}'
        )
    )
layout = {
    'title' : f'Время работы алгоритма от числа типов процессоров',
    'xaxis' : {
        'title':'Число типов процессоров'
    },
    'yaxis' : {
        'title':'Время [ms]'
    }
}
fig = {
    'data':data,
    'layout':layout
}
iplot(fig)
go.Figure(fig).write_image("res4.jpg")

In [43]:
for proc in df_res_g['Система'].unique():
    results = df_res_g[df_res_g['Система'] == proc]
    data = []
    for load in results['Система'].unique():
        df_tmp = results[results['Система'] == load]
        data.append(
            go.Scatter(
                x=df_tmp['Расширенная система'],
                y=df_tmp['Стоимость'],
                name = f'Стоимость {load}'
            )
        )
        data.append(
            go.Scatter(
                x=df_tmp['Расширенная система'],
                y=df_tmp['Ожидаемая стоимость'],
                name = f'Ожидаемая стоимость {load}'
            )
        )
    layout = {
        'title' : f'Время работы алгоритма от числа процессоров для системы из {proc} процессоров',
        'xaxis' : {
            'title':'Число процессоров'
        },
        'yaxis' : {
            'title':'Время [ms]'
        }
    }
    fig = {
        'data':data,
        'layout':layout
    }
    iplot(fig)

# Подобие реальной системы

In [376]:
result = subprocess.call([f'./scheduler', '10', f'tasks/real_task.tsv', f'systems/systems2new' ,'schedule'])

In [377]:
time, performance = get_time_perfomance(f'tasks/real_task.res')

In [378]:
time

434.891998

In [379]:
performance

nan